[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MaxMitre/Redes_Neuronales_Scratch/blob/main/semana6/Solucion_Ejercicio.ipynb)

# Solucion a Ejercicio

Se muestra un objeto de clase red neuronal recurrente que supone que empezamos con una capa LSTM para despues utilizar capas densas en la red neuronal.

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
X_raw = pd.read_csv('/content/drive/MyDrive/Curso-NN-de-Cero/semana5/input_training_IrTAw7w.csv').set_index('ID')
X_raw

In [ ]:
y_raw = pd.read_csv('/content/drive/MyDrive/Curso-NN-de-Cero/semana5/output_training_F2dZW38.csv').set_index('ID')
y_raw

In [ ]:
# Nuestra función de perdida (Error cuadrátio medio)
def mse(y_hat, y):
    return np.mean((y - y_hat)**2)

def d_mse(y_hat, y):
    return (1 / y_hat.shape[0]) * (1/y_hat.shape[-1]) * -2 * np.sum(y - y_hat, axis=0)

In [ ]:
def linear(x):
    return x


def d_linear(x):
    return np.ones_like(x)


def relu(x):
    return np.where(x > 0, x, 0)


def d_relu(x):
    return np.where(x > 0, 1, 0)


def sigmoid(x):
    return 1 / (1 + np.exp(-x))


def d_sigmoid(x):
    return sigmoid(x) * (1 - sigmoid(x))

def softmax(X):
    X_exp = np.exp(X)
    partition = X_exp.sum(1, keepdims=True)
    return X_exp / partition

In [ ]:
np.random.seed(10)

class Red:
    def __init__(self, input_size, lstm, hidden_layers, output_size, hidden_state = 7): # Hidden layers una lista de diccionarios para saber orden
        
        self.input_size = input_size
        
        dense_sizes = [lstm] + hidden_layers + [output_size]
        self.weights = []
        self.biases = []
        for i in range(len(dense_sizes) - 1):
            n_in = dense_sizes[i]
            n_out = dense_sizes[i + 1]
            self.weights.append(np.random.randn(n_in, n_out))
            self.biases.append(np.random.randn(1, n_out))

        W_x = np.random.randn(1, hidden_state)
        W_h = np.random.randn(hidden_state, hidden_state)

        self.weights = [[W_x,W_h]] + self.weights
        self.biases = [np.random.randn(1, hidden_state)] + self.biases

        self.activations = [linear] + [relu for _ in range(len(self.weights) - 1)] + [linear]
        self.d_activations = [d_linear] + [d_relu for _ in range(len(self.weights) - 1)] + [d_linear]

        self.hidden_state = hidden_state
        self.h_0 = np.random.randn(1, hidden_state)


    def forward(self, x):
        h_aux = self.h_0.copy()
        for i in range(self.input_size[0]):
            h_aux = linear(x[i] @ self.weights[0][0] + h_aux @ self.weights[0][1] + self.biases[0])  

        x = h_aux
        for i in range(1, len(self.weights)):

            x_hat = x @ self.weights[i] + self.biases[i]
            x = self.activations[i + 1](x_hat)
        return softmax(x)

In [ ]:
model = Red((5,1), 7, [8,7,7], 3)

In [ ]:
model.weights[0].shape

In [ ]:
model.forward(np.random.randn(5,1))